# classification

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

import cv2 as cv 
from PIL import Image 

import time

#from pycoral.adapters import classify, common
#from pycoral.utils.edgetpu import make_interpreter
#from pycoral.utils.dataset import read_label_file

from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [2]:
reachy = ReachySDK('localhost')

In [17]:
path_model = '/home/reachy/Desktop/quant/model_quant.tflite'
path_label = '/home/reachy/Desktop/quant/quant.txt'

In [ ]:
path_model = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/models/ttt-boxes.tflite'
path_label = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/models/ttt-boxes.txt'


In [ ]:
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

labels = read_label_file(path_label)
print(labels)
size = common.input_size(interpreter)

In [ ]:
board_cases = np.array((
    ((81, 166, 260, 340), #Coordinates first board cases (top-left corner) (Xbl, Xbr, Ytr, Ybr)
     (166, 258, 260, 340), #Coordinates second board cases
     (258, 349, 260, 340),),

    ((74, 161, 340, 432),
     (161, 261, 340, 432),
     (261, 360, 340, 432),),

    ((65, 161, 432, 522),
     (161, 266, 432, 522),
     (266, 365, 432, 522),),
))
# left, right, top, bottom
board_rect = np.array((
    74, 384, 216, 483,
))

In [ ]:
im = reachy.right_camera.wait_for_new_frame()
height, width = np.shape(im)[:2]


In [ ]:
video = cv.VideoWriter('tuto_classification1.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(0.5, 0, -0.5, duration=1.25)
time.sleep(1.25)
start = time.time()

fps_count = 0

while time.time() - start < 15:
    
    im = reachy.right_camera.wait_for_new_frame()
    out = im.copy()

    for row in range(3):
        for col in range(3):
            lx, rx, ly, ry = board_cases[row, col]
            pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
            pil_img.save(f'/home/reachy/Desktop/image{row}{col}.png')
            common.set_input(interpreter, pil_img)
            interpreter.invoke()
            result = classify.get_classes(interpreter, top_k=3, score_threshold=0.8)
            print(f' resultat ={result}')
            if result :
                label = labels.get(result[0].id)
                cv.putText(out, label, (int((lx+rx)/2)-30, int((ly+ry)/2)), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
                cv.rectangle(out, (lx, ly), (rx, ry), (0, 255, 0), 5)
    video.write(out)        
            
video.release()

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)

In [18]:
def load_labels(path):
    with open(path, 'r') as f:
        return {i: line.strip() for i, line in enumerate(f.readlines())}


def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def classify_image(interpreter, image, top_k=3):
    """Returns a sorted array of classification results."""
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    print(f'output = {output}')

    # If the model is quantized (uint8 data), then dequantize the results
    if output_details['dtype'] == np.uint8:
        print('passee')
        scale, zero_point = output_details['quantization']
        output = scale * (output - zero_point)

    ordered = np.argpartition(-output, top_k)
    return [(i, output[i]) for i in ordered[:top_k]]

In [19]:
labels = load_labels(path_label)

interpreter = Interpreter(path_model)
#interpreter = Interpreter(path_model, experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]['shape']

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(0.5, 0, -0.5, duration=1.25)
time.sleep(1.25)
start = time.time()

fps_count = 0
nb=0

while time.time() - start < 15:
    
    im = reachy.right_camera.wait_for_new_frame()
    print(type(im))
    out = im.copy()
    pil_img = Image.fromarray(out).convert('RGB').resize((width, height),Image.ANTIALIAS)
    #pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
    pil_img.save(f'/home/reachy/Desktop/image{nb}.png')
    #common.set_input(interpreter, pil_img)
    #interpreter.invoke()
    results = classify_image(interpreter, pil_img)
    label_id, prob = results[0]
    #result = classify.get_classes(interpreter, top_k=3, score_threshold=0.8)
    print(f' resultat ={results}')
    #print(f' resultat ={labels[label_id]}')
    cv.putText(out,labels[label_id], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
    nb = nb+1

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)

<class 'numpy.ndarray'>
output = [[ -83  -58   48    4]
 [ -49  -71  122   42]
 [ -76  -91   17   34]
 [ -70  -77   57   71]
 [ -97  -87   58   33]
 [ -16 -110   93  -38]
 [ -62  -71   13   60]
 [-128  -22   67   90]
 [ -64 -115   54   20]
 [ -30   -7   99   68]]
 resultat =[(array([2, 3, 1, 0]), array([[-76, -91,  17,  34],
       [-70, -77,  57,  71],
       [-49, -71, 122,  42],
       [-83, -58,  48,   4]], dtype=int8)), (array([2, 3, 0, 1]), array([[-76, -91,  17,  34],
       [-70, -77,  57,  71],
       [-83, -58,  48,   4],
       [-49, -71, 122,  42]], dtype=int8)), (array([2, 3, 0, 1]), array([[-76, -91,  17,  34],
       [-70, -77,  57,  71],
       [-83, -58,  48,   4],
       [-49, -71, 122,  42]], dtype=int8))]


TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
reachy.turn_on('head')

In [ ]:
results[0]

In [ ]:
reachy.head.look_at(1, 0, 0, duration=1.5)